https://pytorch.org/tutorials/beginner/ptcheat.html

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

https://pytorch.org/tutorials/beginner/translation_transformer.html

https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html



<div dir="rtl" lang="he" xml:lang="he">

# אימון מודל טקסט

## התקנת ספריות

נתקין את הספריות הדרושות, רובן כבר מותקנות על _colab_ 
ולא נצטרך להתקין אותם מ_pip_

In [ ]:
%pip install torchdata
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import torchtext
import torch
import torchdata
from torchtext.datasets import AG_NEWS


<div dir="rtl" lang="he" xml:lang="he">

###שימוש במאגרי מידע
`AG_NEWS` 
זה אובייקט איטרבילי, כך שניתן לעבור על הטקסט וסוג החדשות שהוא נותן 



In [ ]:
iter_text = iter(AG_NEWS(split="test"))

print(next(iter_text))
print(next(iter_text))
print(next(iter_text))

<div dir="rtl" lang="he" xml:lang="he">

## יצירת טוקנים
בשלב ראשון נרצה לפצל את הטקסט ליחידות _tokens_ 


In [ ]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')
text = next(iter_text)[1]
print(text)
print(tokenizer(text))

Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.
['prediction', 'unit', 'helps', 'forecast', 'wildfires', '(', 'ap', ')', 'ap', '-', 'it', "'", 's', 'barely', 'dawn', 'when', 'mike', 'fitzpatrick', 'starts', 'his', 'shift', 'with', 'a', 'blur', 'of', 'colorful', 'maps', ',', 'figures', 'and', 'endless', 'charts', ',', 'but', 'already', 'he', 'knows', 'what', 'the', 'day', 'will', 'bring', '.', 'lightning', 'will', 'strike', 'in', 'places', 'he', 'expects', '.', 'winds', 'will', 'pick', 'up', ',', 'moist', 'places', 'will', 'dry', 'and', 'flames', 'will', 'roar', '.']


<div dir="rtl" lang="he" xml:lang="he">

## יצירת מילון
בשביל שהמודל יצליח לעבוד עם טקסט,דבר ראשון נייצג כל מילה בתור מספר. 
לשם כך יש אובייקט מיוחד `vocab` שתפקידו לתת מספר לכל המלים. 
נשים לב שכדאי לתת למילון ערך דיפולטיבי עבור מילים שלא במילון, בשביל שנוכל לנתח מילים שלא מופיעות במילון.

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(iter_text), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


vocab(['unit', 'helps', 'forecast','wordNotApper'])

[772, 2312, 731, 0]

<div dir="rtl" lang="he" xml:lang="he">

# הפיכת מאגר הטקסט לטנזורים
נשתמש באובייקט בשם `DataLoader` 
שעובר על כל הדאטה ומחזיר אותו כקבוצות, _batchs_ 
של טנזורים, אותם נשלח למודל לאימון. 

הפונקציה הזו תחזיר 3 טנזורים:
- `label_list` רשימה של כל התגיות  בקבוצה
- `offsets` רשימה שמציינת איפה מתחיל כל משפט בטקסט
- `text_list` טנזור אחד שמכיל את הטקסטים


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(int(_label)-1)
         processed_text = torch.tensor(
             vocab(tokenizer(_text)), 
             dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) 
    text_list = torch.cat(text_list) 
    return label_list.to(DEVICE), text_list.to(DEVICE), offsets.to(DEVICE) 

<div dir="rtl" lang="he" xml:lang="he">


## הפיכת כל הדאטה לטנזורים
`DataLoader` 
זה אובייקט שאוסף את כל הדאטא וע"י הפונקציה 
`collate_fn` 
הוא הופך את המידע לטנזורים.

In [ ]:
from torch.utils.data import DataLoader
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=2, shuffle=False, collate_fn=collate_batch)
iter_dataloader = iter(dataloader)
print(next(iter_dataloader))
print(next(iter_dataloader))
print(next(iter_dataloader))

<div dir="rtl" lang="he" xml:lang="he">

# יצירת רשת נוירונים

### מרכיבי הרשת
- `embedding` - שכבה שלוקחת כל מילה ומציגה אותה בתור וקטור מספרים, בצורה כזו ניתן להבין אילו מהמילים קרובות זו לזו. במהלך ריצת הרשת השכבה הזו גם מתעדכנת ויכולה להבין אילו מילים רחוקות אחת מהשניה ואלו קרובות
- `Linear` שיכבה ליניארית אחת


In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

<div dir="rtl" lang="he" xml:lang="he">

# אימון המודל

<div dir="rtl" lang="he" xml:lang="he">

### יצירת מודל ופרמטרים

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset



#create model
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64 #size of vector for every word
model = TextClassificationModel(vocab_size, emsize, num_class).to(DEVICE)




<div dir="rtl" lang="he" xml:lang="he">

## יצירת מרכיבים לאימון
בשביל האימון אנחנו צריכים:
- `optimizer` פונקציית אופיטימיזציה שתעדכן את המשקולות במודל
- `criterion` 
- `scheduler` \
חוץ מזה נצטרך היפרפרמטרים:
- `EPOCHS` בכמה מחזורים נעשה את האימון, כל מחזור הוא תהליך אימון ואז הערכה של התוצאות
- `LR` שיקבע לפונקציית אופטימיזציה את גודל הצעד שצריך לעשות כל פעם
- `BATCH_SIZE` קובע בתוך תהליך האימון כל כמה זמן לרוץ ואז לעדכן את הערכים של המודל


In [ ]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

optimizer = torch.optim.SGD(model.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

<div dir="rtl" lang="he" xml:lang="he">

## פונקציית אימון והערכה
ניצור פונקציית אימון שתעבור כל פעם על סט ערכים (

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count
  


In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)